In [ ]:
!pip install numpy matplotlib pillow opencv-python scikit-learn

In [ ]:
import os
import gc
import cv2
import glob
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from collections import Counter
from sklearn.utils import shuffle
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if len(gpus) > 0:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("✅ GPU detectada y configurada con memory_growth")
    except:
        print("⚠️ No se pudo configurar memory_growth en GPU")
    strategy = tf.distribute.MirroredStrategy()
else:
    print("⚠️ No se detectó GPU, usando CPU")
    strategy = tf.distribute.get_strategy()

print(f"Usando estrategia: {type(strategy).__name__}")
print(f"Número de réplicas: {strategy.num_replicas_in_sync}")


✅ GPU detectada y configurada con memory_growth
Usando estrategia: MirroredStrategy
Número de réplicas: 1


In [ ]:
gc.collect()
tf.keras.backend.clear_session()

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [ ]:
!pip install kaggle
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"cherfo17","key":"b71b17e376fd5d1b31ada0103f81e5a1"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d orvile/brain-cancer-mri-dataset

Dataset URL: https://www.kaggle.com/datasets/orvile/brain-cancer-mri-dataset
License(s): CC-BY-SA-4.0
 82% 119M/144M [00:00<00:00, 1.24GB/s]
100% 144M/144M [00:00<00:00, 1.24GB/s]


In [ ]:
zip_file_path = '/content/brain-cancer-mri-dataset.zip'

extracted_folder_path = '/content/Dataset'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [ ]:
imagenes = []
etiquetas = []

In [ ]:
import os
import glob
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf

# 1) Carga y preprocesamiento inicial
carpeta_principal = '/content/Dataset/Brain_Cancer raw MRI data/Brain_Cancer'
imagenes, etiquetas = [], []

# Itera subcarpetas (una por clase)
for z, subcarpeta in enumerate(sorted(glob.glob(os.path.join(carpeta_principal, '*')))):
    if not os.path.isdir(subcarpeta):
        continue
    print(f'Clase {z} → {os.path.basename(subcarpeta)}')
    for ruta in glob.iglob(os.path.join(subcarpeta, '*.jpg')):
        img = cv2.imread(ruta, cv2.IMREAD_GRAYSCALE)        # Lee ya en gris
        img = cv2.resize(img, (512, 512), interpolation=cv2.INTER_AREA)
        imagenes.append(img.astype(np.float32) / 255.0)      # normaliza aquí
        etiquetas.append(z)

imagenes = np.stack(imagenes, axis=0)   # shape: (N, 512, 512)
etiquetas = np.array(etiquetas)         # shape: (N,)

# 2) Barajar y partir (con estratificación)
#    Shuffle explícito antes de split para mayor aleatoriedad
perm = np.random.RandomState(seed=42).permutation(len(imagenes))
imagenes, etiquetas = imagenes[perm], etiquetas[perm]

train_imgs, test_imgs, train_lbls, test_lbls = train_test_split(
    imagenes, etiquetas,
    test_size=0.25,
    random_state=42,
    stratify=etiquetas
)

# 3) Datos para tf.data: añade canal y one-hot
num_classes = len(np.unique(etiquetas))

def make_dataset(images, labels, batch_size=32, shuffle=True):
    # Expande canal y convierte labels a one-hot
    images = images[..., np.newaxis]                        # (B, 512,512,1)
    labels = tf.keras.utils.to_categorical(labels, num_classes)

    ds = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(images), seed=42)
    ds = ds.batch(batch_size)
    ds = ds.cache()                # Opcional, en RAM si te cabe
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

batch_size = 62
train_ds = make_dataset(train_imgs, train_lbls, batch_size=batch_size, shuffle=True)
val_ds   = make_dataset(test_imgs,  test_lbls,  batch_size=batch_size, shuffle=False)

# 4) Prueba rápida: muestra un batch
for x_batch, y_batch in train_ds.take(1):
    print("Shape batch imágenes:", x_batch.shape)   # → (batch_size,512,512,1)
    print("Shape batch labels:", y_batch.shape)     # → (batch_size, num_classes)
    break


Clase 0 → brain_glioma
Clase 1 → brain_menin
Clase 2 → brain_tumor
Shape batch imágenes: (16, 512, 512, 1)
Shape batch labels: (16, 3)


In [ ]:
# Muestra la distribución de etiquetas en el primer batch
for x_batch, y_batch in train_ds.take(1):
    counts = tf.reduce_sum(y_batch, axis=0).numpy()
    print("Instancias por clase en batch:", counts)
    break

Instancias por clase en batch: [3. 8. 5.]


In [ ]:
import os, glob, cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

# --- 1. Aumento de datos ---
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
], name="data_augmentation")

# --- 2. Construcción del modelo ---
def build_high_accuracy_model(input_shape=(512, 512, 1), num_classes=3):
    inputs = layers.Input(shape=input_shape, name="input_gray")
    x = layers.Conv2D(3, kernel_size=3, padding="same", name="to_rgb")(inputs)
    x = data_augmentation(x)

    base_model = tf.keras.applications.MobileNetV3Small(
        input_shape=(512, 512, 3),
        include_top=False,
        weights="imagenet",
        pooling="avg"
    )
    x = base_model(x)

    x = layers.Dense(128, kernel_regularizer=regularizers.l2(1e-4), name="fc1")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="predictions")(x)

    model = models.Model(inputs=inputs, outputs=outputs, name="HighAccNet_GrayTransfer")
    return model

# --- 3. Class Weights ---
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_lbls),
    y=train_lbls
)
class_weight_dict = {i: w for i, w in enumerate(class_weights)}
print("Pesos por clase:", class_weight_dict)

# --- 4. Callbacks ---
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=10,
        restore_best_weights=True
    )
]

# --- 5. Fase 1: Congelar MobileNetV3 ---
model = build_high_accuracy_model()
model.get_layer('MobileNetV3Small').trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\nEntrenando fase 1 (solo capas densas)...")
history_1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks,
    class_weight=class_weight_dict
)

# --- 6. Fase 2: Fine-tune completo ---
model.get_layer('MobileNetV3Small').trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\nEntrenando fase 2 (fine-tuning completo)...")
history_2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=40,
    callbacks=callbacks,
    class_weight=class_weight_dict
)


Pesos por clase: {0: np.float64(1.0073186959414504), 1: np.float64(1.0073186959414504), 2: np.float64(0.9856770833333334)}


/usr/local/lib/python3.11/dist-packages/keras/src/applications/mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(



Entrenando fase 1 (solo capas densas)...
Epoch 1/10
284/284 ━━━━━━━━━━━━━━━━━━━━ 20s 41ms/step - accuracy: 0.6084 - loss: 0.8612 - val_accuracy: 0.3309 - val_loss: 1.9592
Epoch 2/10
284/284 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - accuracy: 0.7046 - loss: 0.6922 - val_accuracy: 0.4293 - val_loss: 2.8812
Epoch 3/10
284/284 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.7120 - loss: 0.6645 - val_accuracy: 0.3309 - val_loss: 13.8703
Epoch 4/10
284/284 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.7250 - loss: 0.6453 - val_accuracy: 0.3514 - val_loss: 3.0545
Epoch 5/10
284/284 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.7575 - loss: 0.6149 - val_accuracy: 0.3309 - val_loss: 9.8887
Epoch 6/10
284/284 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - accuracy: 0.7492 - loss: 0.6130 - val_accuracy: 0.5059 - val_loss: 2.8931
Epoch 7/10
284/284 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - accuracy: 0.7586 - loss: 0.6126 - val_accuracy: 0.6090 - val_loss: 1.0927
Epoch 8/10
284/284 ━━━━━━━━━━━━━━━━━━━━ 10s 34m

In [ ]:
import collections
labels_val = []
for _, y_batch in val_ds:
    labels_val.extend(np.argmax(y_batch.numpy(), axis=1))

print("Distribución en validación:", collections.Counter(labels_val))

Distribución en validación: Counter({np.int64(2): 512, np.int64(1): 501, np.int64(0): 501})


In [ ]:
# --- 7. Evaluación final ---
y_true, y_pred = [], []
for x_batch, y_batch in val_ds:
    preds = model.predict(x_batch, verbose=0)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(np.argmax(y_batch.numpy(), axis=1))

cm = confusion_matrix(y_true, y_pred)
cr = classification_report(y_true, y_pred, target_names=[f"Clase {i}" for i in range(3)])

print("\n🔍 Matriz de confusión:\n", cm)
print("\n📋 Reporte de clasificación:\n", cr)



🔍 Matriz de confusión:
 [[492   3   6]
 [170 174 157]
 [ 53   2 457]]

📋 Reporte de clasificación:
               precision    recall  f1-score   support

     Clase 0       0.69      0.98      0.81       501
     Clase 1       0.97      0.35      0.51       501
     Clase 2       0.74      0.89      0.81       512

    accuracy                           0.74      1514
   macro avg       0.80      0.74      0.71      1514
weighted avg       0.80      0.74      0.71      1514



In [ ]:
model.save('modelo_entrenado.keras')